In [1]:
import sys
import numpy as np
from matplotlib import pyplot as plt
import h5py

%matplotlib notebook

In [2]:
import qcodes as qc

In [107]:
from importlib import reload
from qctools import instruments as instools
reload(instools)

import labpythonconfig as cfg
reload(cfg)

from experiment import measurement
reload(measurement)
from experiment.measurement import Parameter, BaseMeasurement

In [108]:
station = qc.Station()

In [109]:
class DummyMeasurement(BaseMeasurement):
    def __init__(self, station):
        super().__init__(station)
        
        self.add_parameter('start', Parameter)
        self.add_parameter('stop', Parameter)
        self.add_parameter('npts', Parameter)
    
    def measure(self):
        print("Params:", self.start(), self.stop(), self.npts())
        

m = DummyMeasurement(station)
m.start(0)
m.stop(1)
m.npts(11)
m.run()

c:\data\2017-12\2017-12-15\2017-12-15_DummyMeasurement-#0002
Params: 0 1 11


In [81]:
class SimpleMsmt(BaseMeasurement):
    def __init__(self, station, func):
        super().__init__(station)
        self.func = func
        
    def measure(self):
        self.func()
        

f = lambda : True
s = SimpleMsmt(station, f)
s.run()

In [38]:
datatools.get_data_loc('test')

{'data_folder': 'c:\\data\\2017-12\\2017-12-15'}

In [41]:
s = "{n}-{i}-{n}"

In [42]:
s.replace("{n}", 'test')

'test-{i}-test'

In [45]:
"001".zfill(2)

'001'

In [51]:
import re
import os

In [63]:
s = "2017-12-15_Name-#001"
pat = "2017-12-15_Name-{i}"